# 모델 및 관련 객체 로드

In [1]:
import joblib

# 모델과 관련 객체 로드
rf_model = joblib.load('best_rf_model.pkl')
svm_model = joblib.load('best_svm_model.pkl')
vectorizer = joblib.load('vectorizer.pkl')
label_encoder = joblib.load('label_encoder.pkl')

# 데이터 로드 및 전처리

In [5]:
import numpy as np
import pandas as pd
import re

# 업데이트된 고객 지원 데이터 로드
file_path = 'customer_support_data.xlsx'
updated_data = pd.read_excel(file_path)

# 불용어 리스트
stopwords = set(['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다'])

# 텍스트 정규화 함수
def preprocess_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()  # 소문자 변환
    text = text.replace("[^가-ㅎㅏ-ㅣ가-힣 ]","")
    text = text.replace("^ +","")
    text = re.sub(r'\d+', '', text)  # 숫자 제거
    text = re.sub(r'\s+', ' ', text)  # 불필요한 공백 제거
    text = re.sub(r'[^\w\s]', '', text)  # 구두점 제거
    
    text = ' '.join([word for word in text.split() if word not in stopwords])  # 불용어 제거
    return text

# 응대메모 컬럼의 대화내용 전처리
updated_data['processed_text'] = updated_data['응대메모'].apply(preprocess_text)


# 감정예측

In [6]:
# 대화내용을 TF-IDF 벡터로 변환
updated_tfidf = vectorizer.transform(updated_data['processed_text'])

# Random Forest 모델을 사용하여 예측
rf_predictions = rf_model.predict(updated_tfidf)

# SVM 모델을 사용하여 예측
svm_predictions = svm_model.predict(updated_tfidf)

# 다수결 방식으로 결합
ensemble_predictions = np.array([rf_predictions, svm_predictions])
final_predictions = np.apply_along_axis(lambda x: np.bincount(x).argmax(), axis=0, arr=ensemble_predictions)

# 예측된 감정을 원래 레이블로 변환
predicted_emotions = label_encoder.inverse_transform(final_predictions)

# 결과를 데이터프레임에 추가
updated_data['predicted_emotion'] = predicted_emotions

# 결과 확인
print(updated_data[['응대메모', 'predicted_emotion']])


                                                    응대메모 predicted_emotion
0      기능의학 / 안과관련 방송보셨다고 함. 안과질환은 아니고 기능의학의 문제로 생긴 안...                분노
1      빈혈로 인하여 철분주사 가능한지 문의. OOO님 진료 후 가능함 안내 드렸고 검사하...                불안
2      수액 맞고 싶다고 하셨는데 퇴근이 6시여서 내원 어렵다고 말씀하셔서 수요일 야간진료...                불안
3                                        방송/재진/진료/장이 불편함                상처
4      방송보고 전화주심/대학병원이냐고 문의하셔서 아니라고 안내드리고 당뇨약처방 원하셔서 ...                상처
...                                                  ...               ...
11877                                  삭센다 1개 드림 / 보관중없음                상처
11878              11월 말 전에 2번정도 오시면 동일 수액 놔주시면 됩니다 -dr최                기쁨
11879                                    OOO님개 드림/ 보관중없음                상처
11880                                          결과확인리콜/문자                상처
11881                                          결과확인리콜/문자                상처

[11882 rows x 2 columns]


# 결과확인

In [7]:
# 결과를 엑셀 파일로 저장
output_file_path = 'customer_support_data_predicted.xlsx'
updated_data.to_excel(output_file_path, index=False)
print(f"예측 결과가 '{output_file_path}'에 저장되었습니다.")

예측 결과가 'customer_support_data_predicted.xlsx'에 저장되었습니다.
